# 04 - dataset blancement

In [ ]:
# Importing libraries

import os
import random
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from shutil import copy2, rmtree
from scipy.ndimage import zoom
from typing import Dict, List, Tuple, Union

In [ ]:
# Utils functions 

def get_water_percentage(dir_name: str) -> int:
    """
    Extract the water percentage from the directory name.

    Args:
        dir_name (str): The name of the directory containing water percentage information.

    Returns:
        int: The water percentage extracted from the directory name.
    """
    return int(dir_name.split('_')[2])
